In [1]:
# unet
import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(
                scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(
                in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = torch.tensor([x2.size()[2] - x1.size()[2]])
        diffX = torch.tensor([x2.size()[3] - x1.size()[3]])
        # import pdb; pdb.set_trace()

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        """softmax"""
        x = self.conv(x)
        # x = F.softmax(x, dim=1)
        return x


class UNet(nn.Module):
    def __init__(self, n_channels=1, n_classes=2, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 32)
        self.down1 = Down(32, 64)
        self.down2 = Down(64, 128)
        self.down3 = Down(128, 256)
        self.down4 = Down(256, 256)
        self.up1 = Up(512, 128, bilinear)
        self.up2 = Up(256, 64, bilinear)
        self.up3 = Up(128, 32, bilinear)
        self.up4 = Up(64, 32, bilinear)
        self.outpred = OutConv(32, n_classes)

    def forward(self, x):
        x = x/255.0
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outpred(x)
        return logits


In [2]:
# data
import os
import gc

import cv2
import numpy as np
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

data_transforms = transforms.Compose([
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])


def load_file(filename):
    img = cv2.imread(filename, -1)
    img = img.astype(np.float32)
    return img


class CellDataset(Dataset):
    def __init__(self, txtpath, transform=data_transforms, data_reader=None):
        super(CellDataset, self).__init__()

        data_paths = []
        with open(txtpath, 'r') as fh:
            for line in fh:
                line = line.strip('\n')
                line = line.rstrip('\n')
                words = line.split()    # 0和1分别是cell和mask路径
                data_paths.append((words[0], words[1]))

        self.data_paths = data_paths
        self.transform = transform
        self.data_reader = data_reader
        pass

    def __getitem__(self, index):

        cell_path, mask_path = self.data_paths[index]
        cell = self.data_reader(cell_path)
        mask = self.data_reader(mask_path)

        # Normalization
        cell = cell - cell.min()
        cell = cell / cell.max() * 255

        if self.transform is not None:
            img = np.uint8([cell, mask, mask]).transpose(1, 2, 0)
            img = Image.fromarray(img)
            img = self.transform(img)
            cell = img[0]
            mask = img[1] * 255

        return cell, mask

    def __len__(self):
        return len(self.data_paths)


def get_dataset(cell_dir, mask_dir, valid_rate, tmp_dir, use_exist=True):

    valid_txt = tmp_dir + "valid_data.txt"
    train_txt = tmp_dir + "train_data.txt"

    use_exist = use_exist and os.path.isfile(
        valid_txt) and os.path.isfile(train_txt)

    if not use_exist:
        # generate list of file names
        cell_list = [os.path.join(cell_dir, image)
                     for image in os.listdir(cell_dir)]
        mask_list = [os.path.join(mask_dir, image)
                     for image in os.listdir(mask_dir)]

        # separate the lists according to valid_rate
        sample_size = len(cell_list)
        valid_size = int(sample_size * valid_rate)
        valid_index = np.random.choice(
            a=sample_size, size=valid_size, replace=False, p=None)

        # save the lists in txt files
        with open(valid_txt, "a+") as f:
            for i in valid_index:
                f.write(cell_list[i] + " " + mask_list[i] + '\n')

        with open(train_txt, "a+") as f:
            for i in range(sample_size):
                if i not in valid_index:
                    f.write(cell_list[i] + " " + mask_list[i] + '\n')

    # get the Dataset objects
    train_dataset = CellDataset(train_txt, data_reader=load_file)
    valid_dataset = CellDataset(valid_txt, data_reader=load_file)

    return train_dataset, valid_dataset


In [3]:
# validate
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy

SHOW_PIC = False


class Validator():

    def __init__(self, unet,
                 hyper_params,
                 use_cuda,
                 data_loader):
        self.unet = unet
        self.hyper_params = hyper_params
        self.use_cuda = use_cuda
        self.data_loader = data_loader
        pass

    def validate(self):
        width_out = 628
        height_out = 628
        batch_size = self.hyper_params["batch_size"]
        use_cuda = self.use_cuda

        j_scores = []
        for i, data in enumerate(self.data_loader):
            # preprocess
            b_val_x, b_val_y = data
            if (len(b_val_x.size()) == 3):
                b_val_x = b_val_x.unsqueeze(1)
            if use_cuda:
                b_val_x = b_val_x.cuda()

            # get predict
            b_predict_y = self.unet(b_val_x)

            # post process
            if use_cuda:
                b_predict_y = b_predict_y.cpu().detach().numpy()
                b_predict_y = self.post_process(b_predict_y)
                b_predict_y = b_predict_y.cuda()

            if SHOW_PIC and i == 0:
                b_val_x = b_val_x.cpu().detach().numpy()
                b_predict_y = b_predict_y.cpu().detach().numpy()
                self.show_pic(b_val_x[0][0], b_val_y[0], b_predict_y[0])

            # calc jaccard score
            for j in range(len(b_predict_y)):
                j_score = self.calc_jaccard(
                    b_predict_y[j], b_val_y[j], use_cuda=self.use_cuda)
                j_scores.append(j_score)

        # print("j_scores:", np.array(j_scores))
        j_score = np.mean(j_scores)
        return j_score

    def post_process(self, batch_predict_y):
        """post process of the result"""
        # shape: [batch_size, 2, width, height]

        batch_predict_y = batch_predict_y[:, 1, :, :]

        res = []
        for predict_y in batch_predict_y:
            # binarization
            predict_y[predict_y > 0] = 1
            predict_y[predict_y <= 0] = 0

            # open
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
            predict_y = cv2.erode(predict_y, kernel)  # 腐蚀
            predict_y = cv2.dilate(predict_y, kernel)  # 膨胀

            # parse
            predict_y = predict_y.astype(np.uint8) * 255
            if cv2.__version__[0] == '3':
                __, contours, _ = cv2.findContours(
                    predict_y, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  # 寻找连通域
            elif cv2.__version__[0] == '4':
                contours, _ = cv2.findContours(
                    predict_y, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  # 寻找连通域

            areas = [cv2.contourArea(cnt) for cnt in contours]
            cellIndexs = np.argsort(areas)

            predict_y = np.zeros([predict_y.shape[0], predict_y.shape[1]])
            for j in range(len(cellIndexs)):
                cv2.drawContours(predict_y, contours, j, j, cv2.FILLED)

            predict_y = predict_y.astype(int)
            res.append(predict_y)

        res = torch.Tensor(res)
        return res

    def calc_jaccard(self, imgA, imgB, use_cuda=True):
        """calculate the jaccard score"""
        """all this may occur in GPU."""

        unqA = torch.unique(imgA)
        unqB = torch.unique(imgB)
        num_A = len(unqA)
        num_B = len(unqB)

        if num_A < num_B:
            imgA, imgB = imgB, imgA
            num_A, num_B = num_B, num_A
            unqA, unqB = unqB, unqA

        for i in range(num_A):
            imgA[imgA == unqA[i]] = i
        for i in range(num_B):
            imgB[imgB == unqB[i]] = i

        hit_matrix = np.zeros([num_A, num_B])

        if use_cuda:
            for i in range(2, num_A):
                A_chan = (imgA == i).cuda()
                for j in range(1, num_B):
                    B_chan = (imgB == j).cuda()
                    A_and_B = torch.mul(A_chan, B_chan)
                    B_chan[A_chan == 1] = 1
                    hit_matrix[i, j] = torch.sum(
                        A_and_B).float() / torch.sum(B_chan).float()
        else:
            for i in range(2, num_A):
                A_chan = (imgA == i)
                for j in range(1, num_B):
                    B_chan = (imgB == j)
                    A_and_B = torch.mul(A_chan, B_chan)
                    B_chan[A_chan == 1] = 1
                    hit_matrix[i, j] = torch.sum(
                        A_and_B).float() / torch.sum(B_chan).float()

        jaccard_list = []
        for j in range(1, num_B):
            jac_col = np.max(hit_matrix[:, j])
            jaccard_list.append(jac_col)

        j_score = np.sum(jaccard_list) / max(num_A, num_B)
        return j_score

    def show_pic(self, picA, picB, picC=None,
                 A_gray=True):
        plt.subplot(1, 3, 1)
        plt.title("x")
        if A_gray:
            plt.imshow(picA, cmap='gray')
        else:
            plt.imshow(picA)

        plt.subplot(1, 3, 2)
        plt.title("GT")
        plt.imshow(picB)

        if picC is not None:
            plt.subplot(1, 3, 3)
            plt.title("Predict")
            plt.imshow(picC)

        plt.show()


In [5]:
# train
import os
import gc

import cv2
import numpy as np
import scipy
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader



SHOW_NET = False



class Trainer():
    def __init__(self):
        pass

    def setup(self, valid_rate=0.1, use_cuda=True,
              cell_dir="", mask_dir="", module_save_dir="", tmp_dir="",
              criterion=None, optimizer=None, hyper_params=None,
              ):
        """setup the module"""
        self.train_dataset, self.valid_dataset = get_dataset(
            cell_dir, mask_dir, valid_rate, tmp_dir)

        self.hyper_params = hyper_params
        self.train_data_loader = DataLoader(
            dataset=self.train_dataset,
            num_workers=self.hyper_params["threads"],
            batch_size=self.hyper_params["batch_size"],
            shuffle=True
        )
        self.valid_data_loader = DataLoader(
            dataset=self.valid_dataset,
            num_workers=self.hyper_params["threads"],
            batch_size=self.hyper_params["batch_size"],
            shuffle=False
        )

        self.use_cuda = use_cuda
        self.unet = UNet(n_channels=1, n_classes=2,)
        if use_cuda:
            self.unet = self.unet.cuda()
        if SHOW_NET:
            from torchsummary import summary
            batch_size = self.hyper_params["batch_size"]
            summary(self.unet, (batch_size, 628, 628))

        self.criterion = torch.nn.CrossEntropyLoss()
        self.optimizer = torch.optim.SGD(
            self.unet.parameters(), lr=self.hyper_params["learning_rate"], momentum=0.99)
        self.module_save_dir = module_save_dir

        self.v = Validator(unet=self.unet,
                           hyper_params=hyper_params,
                           use_cuda=use_cuda,
                           data_loader=self.valid_data_loader)

    def train(self):
        """train the model"""
        epochs = self.hyper_params["epochs"]
        epoch_lapse = self.hyper_params["epoch_lapse"]
        batch_size = self.hyper_params["batch_size"]
        epoch_save = self.hyper_params["epoch_save"]
        width_out = 628
        height_out = 628

        accs = []
        for _ in range(epochs):
            total_loss = 0
            for data in tqdm(self.train_data_loader, ascii=True, ncols=120):

                batch_train_x, batch_train_y = data
                batch_train_y = batch_train_y.long()
                batch_train_y[batch_train_y > 0] = 1  # important!!!
                if (len(batch_train_x.size()) == 3):
                    batch_train_x = batch_train_x.unsqueeze(1)
                if (len(batch_train_y.size()) == 3):
                    batch_train_y = batch_train_y.unsqueeze(1)

                if self.use_cuda:
                    batch_train_x = batch_train_x.cuda()
                    batch_train_y = batch_train_y.cuda()

                batch_loss = self.train_step(
                    batch_train_x, batch_train_y,
                    optimizer=self.optimizer,
                    criterion=self.criterion,
                    unet=self.unet,
                    width_out=width_out,
                    height_out=height_out,
                    batch_size=batch_size)

                total_loss += batch_loss

            if (_+1) % epoch_lapse == 0:
                val_acc = self.v.validate()
                print("Total loss in epoch %f : %f and validation accuracy : %f" %
                      (_ + 1, total_loss, val_acc))
                accs.append(val_acc)

            if (_+1) % epoch_save == 0:
                self.save_module(name_else="epoch-" + str(_+1))
        print(accs)
        gc.collect()
        pass

    def train_step(self, inputs, labels, optimizer,
                   criterion, unet, batch_size,
                   width_out, height_out):
        optimizer.zero_grad()
        outputs = unet(inputs)
        outputs = outputs.permute(0, 2, 3, 1)

        outputs = outputs.reshape(batch_size * width_out * height_out, 2)
        labels = labels.reshape(batch_size * width_out * height_out)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        return loss

    def save_module(self, name_else=""):
        import datetime
        module_save_dir = self.module_save_dir
        filename = 'unet-' + datetime.datetime.now().strftime('%Y%m%d%H%M%S') + \
            name_else + '.pth'
        torch.save(self.unet.state_dict(), module_save_dir + filename)
        pass


In [6]:
hyper_parameters = {
            "batch_size": 2,
            "learning_rate": 1e-4,
            "threads": 0,
            "epochs": 5000,
            "epoch_lapse": 50,
            "epoch_save": 100,
        }

cell_dir = "C:/me/dataset1/train/"
mask_dir = "C:/me/dataset1/train_GT/SEG"
module_save_dir = "C:/me/test/save/"
tmp_dir = "C:/me/test/_tmp/"

valid_rate = 0.1
use_cuda = True
trainer = Trainer()

In [7]:
trainer.setup(cell_dir=cell_dir,
              mask_dir=mask_dir,
              module_save_dir=module_save_dir,
              tmp_dir=tmp_dir,
              valid_rate=valid_rate,
              hyper_params=hyper_parameters,
              use_cuda=use_cuda)

In [8]:
trainer.train()

  0%|                                                                                            | 0/79 [00:00<?, ?it/s]

Total loss in epoch 50.000000 : 3.650290 and validation accuracy : 0.562703


 78%|#################################################################1                 | 62/79 [00:11<00:03,  5.25it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  0%|                                                                                            | 0/79 [00:00<?, ?it/s]

Total loss in epoch 150.000000 : 2.893389 and validation accuracy : 0.598361


 96%|###############################################################################8   | 76/79 [00:14<00:00,  5.02it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  0%|                                                                                            | 0/79 [00:00<?, ?it/s]

...

Total loss in epoch 1850.000000 : 0.961668 and validation accuracy : 0.608924


 90%|##########################################################################5        | 71/79 [00:13<00:01,  5.24it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  0%|                                                                                            | 0/79 [00:00<?, ?it/s]

Total loss in epoch 1950.000000 : 0.944577 and validation accuracy : 0.621619


 34%|############################3                                                      | 27/79 [00:05<00:10,  5.17it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  4%|###1                                                                                | 3/79 [00:00<00:18,  4.18it/s]


KeyboardInterrupt: 

In [8]:
import datetime
trainer.save_module()